In [1]:
import energy_meters.shelly as shelly

shelly_df = shelly.process_energy_history(debug=True)
shelly_df.tail()

98.9kiB [00:01, 83.5kiB/s]



Download completed. File saved as /workspace/data/shelly/em_data.grid.csv



98.3kiB [00:01, 85.0kiB/s]



Download completed. File saved as /workspace/data/shelly/em_data.solar.csv
Getting energy data...
Resampling data...
Selecting columns...
Converting units...
Renaming columns...
Rounding values...
Exporting data...


,grid_kWh,solar_kWh,consumed_kWh
timestamp_utc,,,
2024-04-21 06:45:00+00:00,0.05648,0.02755,0.08403
2024-04-21 07:00:00+00:00,0.01129,0.04398,0.05527
2024-04-21 07:15:00+00:00,-0.00751,0.06344,0.05593
2024-04-21 07:30:00+00:00,0.02092,0.08428,0.1052
2024-04-21 07:45:00+00:00,-0.01964,0.03294,0.0133


In [ ]:
import providers.repsol as repsol

repsol_prices_df = repsol.get_latest_prices()

In [ ]:
costs_df = shelly_df.join(repsol_prices_df, how="inner")
costs_df["bought_€"] = (costs_df["€/kWh"] * costs_df["main_kWh"]).clip(lower=0)
costs_df["lost_€"] = -(costs_df["€/kWh"] * costs_df["main_kWh"]).clip(upper=0)
costs_df["solar_production_€"] = costs_df["€/kWh"] * costs_df["solar_kWh"]
costs_df["solar_consumtion_€"] = costs_df["solar_production_€"] - costs_df["lost_€"]
costs_df["consumed_€"] = costs_df["€/kWh"] * costs_df["consumed_kWh"]
costs_df[
    [
        "consumed_€",
        "bought_€",
        "solar_consumtion_€",
        "solar_production_€",
        "lost_€",
    ]
].resample("1h").sum().head()

In [ ]:
costs_df[
    [
        "consumed_€",
        "bought_€",
        "solar_consumtion_€",
        "solar_production_€",
        "lost_€",
    ]
].resample("1D").sum()

In [ ]:
current_month_consumption_history_df = pd.read_csv(
    "/workspace/data/current_month_consumption_history.csv",
    sep=",",
    header=0,
    index_col=0,
    names=["timestamp_utc", "Consumption (kWh)", "Injection (kWh)"],
    dtype={"Consumption (kWh)": "Float64", "Injection (kWh)": "Float64"},
    parse_dates=["timestamp_utc"],
)
shelly_consumption_history_df = pd.read_csv(
    "/workspace/data/shelly_consumption_history.csv",
    sep=",",
    header=0,
    index_col=0,
    names=["timestamp_utc", "Grid (kWh)", "Solar (kWh)", "Consumed (kWh)"],
    dtype={
        "Grid (kWh)": "Float64",
        "Solar (kWh)": "Float64",
        "Consumed (kWh)": "Float64",
    },
    parse_dates=["timestamp_utc"],
)

compare_df = pd.merge(
    current_month_consumption_history_df,
    shelly_consumption_history_df,
    left_index=True,
    right_index=True,
    how="inner",
)

compare_df = compare_df.resample("1h").sum()
compare_df.loc[:, "E-Redes (kWh)"] = (
    compare_df.loc[:, "Consumption (kWh)"] - compare_df.loc[:, "Injection (kWh)"]
)
compare_df.loc[:, "Shelly (kWh)"] = compare_df.loc[:, "Grid (kWh)"]
compare_df.loc[:, "Diff (kWh)"] = (
    compare_df.loc[:, "Grid (kWh)"]
    - compare_df.loc[:, "Consumption (kWh)"]
    + compare_df.loc[:, "Injection (kWh)"]
)
print(
    f'Desvio médio de {compare_df["Diff (kWh)"].mean()*1000:.2f} Wh em relação ao consumo real.'
)
print(
    f'Desvio máximo de {compare_df["Diff (kWh)"].max()*1000:.2f} Wh em relação ao consumo real.'
)
# compare_df[
#     [
#         "E-Redes (kWh)",
#         "Shelly (kWh)",
#         "Diff (kWh)",
#     ]
# ].plot.line(figsize=(15, 5))

In [ ]:
import plot

plot.weekly_energy_consumption()